In [1]:
import os
import numpy as np
from data_loader import DataLoader
import pickle

In [2]:
root = os.getcwd()
X = np.load(root+"/../datasets/states.npy")
A = np.load(root+"/../datasets/actions.npy")
Y = np.load(root+"/../datasets/next_states.npy")
X_cat = np.concatenate((X, A), axis=2)

In [3]:
print(X.shape, A.shape, X_cat.shape, Y.shape)

(178527, 100, 21) (178527, 100, 8) (178527, 100, 29) (178527, 100, 21)


In [4]:
X.shape[0]//100

1785

In [5]:
input_states = {}
input_actions = {}
intermediate_states = {}
output_actions = {}
output_states = {}

for n in range(X.shape[0]):
    x = X[n]        # shape (T=100, D=21)
    a = A[n]        # shape (T=100, D=8)
    for t in range(1, x.shape[0]-1):
        input_state_sequence = x[0:t]
        input_action_sequence = a[0:t]
        intermediate_state = x[t]
        output_state_sequence = x[t+1:]
        output_action_sequence = a[t:-1]
        
        if t in input_states.keys():
            input_states[t].append(input_state_sequence)
            input_actions[t].append(input_action_sequence)
            intermediate_states[t].append(intermediate_state)
            output_actions[t].append(output_action_sequence)
            output_states[t].append(output_state_sequence)
        else:
            input_states[t] = [input_state_sequence]
            input_actions[t] = [input_action_sequence]
            intermediate_states[t] = [intermediate_state]
            output_actions[t] = [output_action_sequence]
            output_states[t] = [output_state_sequence]

for t in input_states.keys():
    input_states[t] = np.array(input_states[t])
    input_actions[t] = np.array(input_actions[t])
    intermediate_states[t] = np.array(intermediate_states[t])
    output_actions[t] = np.array(output_actions[t])
    output_states[t] = np.array(output_states[t])

MemoryError: Unable to allocate 1.24 GiB for an array with shape (178527, 89, 21) and data type float32

In [9]:
for t in input_states.keys():
    print(t, input_states[t].shape, input_actions[t].shape, intermediate_states[t].shape, output_states[t].shape, output_actions[t].shape)

1 (1785, 1, 21) (1785, 1, 8) (1785, 21) (1785, 98, 21) (1785, 98, 8)
2 (1785, 2, 21) (1785, 2, 8) (1785, 21) (1785, 97, 21) (1785, 97, 8)
3 (1785, 3, 21) (1785, 3, 8) (1785, 21) (1785, 96, 21) (1785, 96, 8)
4 (1785, 4, 21) (1785, 4, 8) (1785, 21) (1785, 95, 21) (1785, 95, 8)
5 (1785, 5, 21) (1785, 5, 8) (1785, 21) (1785, 94, 21) (1785, 94, 8)
6 (1785, 6, 21) (1785, 6, 8) (1785, 21) (1785, 93, 21) (1785, 93, 8)
7 (1785, 7, 21) (1785, 7, 8) (1785, 21) (1785, 92, 21) (1785, 92, 8)
8 (1785, 8, 21) (1785, 8, 8) (1785, 21) (1785, 91, 21) (1785, 91, 8)
9 (1785, 9, 21) (1785, 9, 8) (1785, 21) (1785, 90, 21) (1785, 90, 8)
10 (1785, 10, 21) (1785, 10, 8) (1785, 21) (1785, 89, 21) (1785, 89, 8)
11 (1785, 11, 21) (1785, 11, 8) (1785, 21) (1785, 88, 21) (1785, 88, 8)
12 (1785, 12, 21) (1785, 12, 8) (1785, 21) (1785, 87, 21) (1785, 87, 8)
13 (1785, 13, 21) (1785, 13, 8) (1785, 21) (1785, 86, 21) (1785, 86, 8)
14 (1785, 14, 21) (1785, 14, 8) (1785, 21) (1785, 85, 21) (1785, 85, 8)
15 (1785, 15, 21) (

In [11]:
def get_split_indices(N, train=0.9, val=0.05, test=0.05):
    n = np.arange(N)
    np.random.shuffle(n)
    print(n)
    a = int(N*train)
    b = int(N*val)
    c = int(N*test)
    train = n[0:a]
    val = n[a:a+b]
    test = n[a+b:]
    return train, val, test

In [13]:
train_i, val_i, test_i = get_split_indices(N=input_states[1].shape[0])

[1502 1282 1483 ...  275  210  493]


In [15]:
train_i.shape[0], val_i.shape[0], test_i.shape[0]

(1606, 89, 90)

In [19]:
input_states_train = {}
input_actions_train = {}
intermediate_states_train = {}
output_states_train = {}
output_actions_train = {}

for t in input_states.keys():
    input_states_train[t] = input_states[t][train_i]
    input_actions_train[t] = input_actions[t][train_i]
    intermediate_states_train[t] = intermediate_states[t][train_i]
    output_states_train[t] = output_states[t][train_i]
    output_actions_train[t] = output_actions[t][train_i]

In [20]:
input_states_val = {}
input_actions_val = {}
intermediate_states_val = {}
output_states_val = {}
output_actions_val = {}

for t in input_states.keys():
    input_states_val[t] = input_states[t][val_i]
    input_actions_val[t] = input_actions[t][val_i]
    intermediate_states_val[t] = intermediate_states[t][val_i]
    output_states_val[t] = output_states[t][val_i]
    output_actions_val[t] = output_actions[t][val_i]

In [21]:
input_states_test = {}
input_actions_test = {}
intermediate_states_test = {}
output_states_test = {}
output_actions_test = {}

for t in input_states.keys():
    input_states_test[t] = input_states[t][test_i]
    input_actions_test[t] = input_actions[t][test_i]
    intermediate_states_test[t] = intermediate_states[t][test_i]
    output_states_test[t] = output_states[t][test_i]
    output_actions_test[t] = output_actions[t][test_i]

In [22]:
train = {
    "input_states" : input_states_train,
    "input_actions": input_actions_train,
    "intermediate_states": intermediate_states_train,
    "output_states": output_states_train,
    "output_actions": output_actions_train
}
val = {
    "input_states" : input_states_val,
    "input_actions": input_actions_val,
    "intermediate_states": intermediate_states_val,
    "output_states": output_states_val,
    "output_actions": output_actions_val
}
test = {
    "input_states" : input_states_test,
    "input_actions": input_actions_test,
    "intermediate_states": intermediate_states_test,
    "output_states": output_states_test,
    "output_actions": output_actions_test
}

In [28]:
with open(root+"/../datasets/custom_rnn_uniform/train.pickle", "wb") as f:
    pickle.dump(train, f)
with open(root+"/../datasets/custom_rnn_uniform/val.pickle", "wb") as f:
    pickle.dump(val, f)
with open(root+"/../datasets/custom_rnn_uniform/test.pickle", "wb") as f:
    pickle.dump(test, f)